In [ ]:
# ===================================================================
# DATASET FILTERING PIPELINE
# Keep only relevant placement emails with extracted features
# ===================================================================

import pandas as pd
import numpy as np
import logging
from typing import List

# ===================================================================
# LOGGING SETUP
# ===================================================================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger("DatasetFilter")

# ===================================================================
# CONFIGURATION
# ===================================================================

# Columns to keep in final dataset
FINAL_COLUMNS = [
    'MessageId',
    'Sender', 
    'Subject',
    'Date',
    'PDFs',
    'combined_text',
    'cleaned_text',
    'processing_status',
    'companies_extracted',
    'skills_extracted',
    'positions_extracted',
    'locations_extracted',
    'salary_info',
    'experience_required',
    'degrees_required'
]

# ===================================================================
# FILTERING CRITERIA
# ===================================================================

def is_relevant_email(row) -> bool:
    """
    Determine if email is relevant based on extracted features.
    
    An email is relevant if it has:
    - At least one company OR
    - At least one skill OR
    - At least one position OR
    - At least one location
    
    This ensures we only keep placement-related emails.
    """
    
    # Check if any feature is extracted
    has_company = pd.notna(row['companies_extracted']) and row['companies_extracted'].strip() != ''
    has_skills = pd.notna(row['skills_extracted']) and row['skills_extracted'].strip() != ''
    has_position = pd.notna(row['positions_extracted']) and row['positions_extracted'].strip() != ''
    has_location = pd.notna(row['locations_extracted']) and row['locations_extracted'].strip() != ''
    has_salary = pd.notna(row['salary_info']) and row['salary_info'].strip() != ''
    has_experience = pd.notna(row['experience_required']) and row['experience_required'].strip() != ''
    has_degree = pd.notna(row['degrees_required']) and row['degrees_required'].strip() != ''
    
    # Email is relevant if it has at least 2 features
    feature_count = sum([
        has_company, 
        has_skills, 
        has_position, 
        has_location,
        has_salary,
        has_experience,
        has_degree
    ])
    
    # Also ensure cleaned text is not empty
    has_content = pd.notna(row['cleaned_text']) and len(str(row['cleaned_text']).strip()) > 20
    
    return feature_count >= 2 and has_content

# ===================================================================
# MAIN FILTERING FUNCTION
# ===================================================================

def filter_relevant_emails(
    input_csv: str = "ai_cleaned_emails.csv",
    output_csv: str = "relevant_placement_emails.csv"
) -> pd.DataFrame:
    """
    Filter dataset to keep only relevant placement emails.
    
    Args:
        input_csv: Path to input CSV (output from AI cleaning pipeline)
        output_csv: Path to save filtered CSV
    
    Returns:
        Filtered DataFrame
    """
    
    logger.info("="*70)
    logger.info("DATASET FILTERING PIPELINE")
    logger.info("="*70)
    
    # Load dataset
    logger.info(f"Loading dataset: {input_csv}")
    df = pd.read_csv(input_csv)
    original_count = len(df)
    logger.info(f"Loaded {original_count} emails")
    
    # Check which columns exist
    logger.info(f"\nAvailable columns: {len(df.columns)}")
    existing_columns = df.columns.tolist()
    
    # Verify required columns exist
    missing_columns = [col for col in FINAL_COLUMNS if col not in existing_columns]
    if missing_columns:
        logger.warning(f"Missing columns: {missing_columns}")
        logger.info(f"Available columns: {existing_columns}")
        # Use only available columns
        final_columns = [col for col in FINAL_COLUMNS if col in existing_columns]
    else:
        final_columns = FINAL_COLUMNS
    
    logger.info(f"Using {len(final_columns)} columns")
    
    # Apply relevance filter
    logger.info(f"\nApplying relevance filter...")
    logger.info(f"   Criteria: At least 2 features extracted + meaningful content")
    
    df['is_relevant'] = df.apply(is_relevant_email, axis=1)
    relevant_df = df[df['is_relevant']].copy()
    relevant_count = len(relevant_df)
    
    # Calculate statistics
    filtered_count = original_count - relevant_count
    retention_rate = (relevant_count / original_count) * 100
    
    logger.info(f"\nFILTERING RESULTS:")
    logger.info(f"   Original emails: {original_count}")
    logger.info(f"   Relevant emails: {relevant_count}")
    logger.info(f"   Filtered out: {filtered_count}")
    logger.info(f"   Retention rate: {retention_rate:.1f}%")
    
    # Show breakdown by feature
    logger.info(f"\nFEATURE BREAKDOWN (Relevant Emails Only):")
    
    with_companies = relevant_df['companies_extracted'].notna() & (relevant_df['companies_extracted'] != '')
    with_skills = relevant_df['skills_extracted'].notna() & (relevant_df['skills_extracted'] != '')
    with_positions = relevant_df['positions_extracted'].notna() & (relevant_df['positions_extracted'] != '')
    with_locations = relevant_df['locations_extracted'].notna() & (relevant_df['locations_extracted'] != '')
    with_salary = relevant_df['salary_info'].notna() & (relevant_df['salary_info'] != '')
    with_experience = relevant_df['experience_required'].notna() & (relevant_df['experience_required'] != '')
    with_degrees = relevant_df['degrees_required'].notna() & (relevant_df['degrees_required'] != '')
    
    logger.info(f"With Companies: {with_companies.sum()} ({with_companies.sum()/relevant_count*100:.1f}%)")
    logger.info(f"With Skills: {with_skills.sum()} ({with_skills.sum()/relevant_count*100:.1f}%)")
    logger.info(f"With Positions: {with_positions.sum()} ({with_positions.sum()/relevant_count*100:.1f}%)")
    logger.info(f"With Locations: {with_locations.sum()} ({with_locations.sum()/relevant_count*100:.1f}%)")
    logger.info(f"With Salary: {with_salary.sum()} ({with_salary.sum()/relevant_count*100:.1f}%)")
    logger.info(f"With Experience: {with_experience.sum()} ({with_experience.sum()/relevant_count*100:.1f}%)")
    logger.info(f"With Degrees: {with_degrees.sum()} ({with_degrees.sum()/relevant_count*100:.1f}%)")
    
    # Select only required columns
    relevant_df = relevant_df[final_columns].copy()
    
    # Sort by number of features (most complete emails first)
    logger.info(f"\nSorting by completeness (most features first)...")
    
    relevant_df['feature_score'] = (
        relevant_df['companies_extracted'].notna().astype(int) +
        relevant_df['skills_extracted'].notna().astype(int) +
        relevant_df['positions_extracted'].notna().astype(int) +
        relevant_df['locations_extracted'].notna().astype(int) +
        relevant_df['salary_info'].notna().astype(int) +
        relevant_df['experience_required'].notna().astype(int) +
        relevant_df['degrees_required'].notna().astype(int)
    )
    
    relevant_df = relevant_df.sort_values('feature_score', ascending=False)
    relevant_df = relevant_df.drop('feature_score', axis=1)
    
    # Save filtered dataset
    logger.info(f"\nSaving filtered dataset...")
    relevant_df.to_csv(output_csv, index=False)
    logger.info(f"Saved to: {output_csv}")
    
    # Show sample of filtered emails
    logger.info(f"\nSAMPLE OF TOP 5 RELEVANT EMAILS:")
    logger.info("="*70)
    
    for idx, row in relevant_df.head(5).iterrows():
        logger.info(f"\n--- Email {idx} ---")
        logger.info(f"Subject: {str(row['Subject'])[:60]}")
        logger.info(f"Companies: {str(row['companies_extracted'])[:60]}")
        logger.info(f"Skills: {str(row['skills_extracted'])[:60]}")
        logger.info(f"Positions: {str(row['positions_extracted'])[:60]}")
        logger.info(f"Locations: {str(row['locations_extracted'])[:60]}")
    
    logger.info(f"\n{'='*70}")
    logger.info(f"FILTERING COMPLETE!")
    logger.info(f"{'='*70}")
    logger.info(f"Original file: {input_csv} ({original_count} emails)")
    logger.info(f"Filtered file: {output_csv} ({relevant_count} emails)")
    logger.info(f"{'='*70}\n")
    
    return relevant_df

# ===================================================================
# ADVANCED FILTERING OPTIONS
# ===================================================================

def filter_by_custom_criteria(
    input_csv: str = "ai_cleaned_emails.csv",
    output_csv: str = "custom_filtered_emails.csv",
    min_companies: int = 0,
    min_skills: int = 1,
    min_positions: int = 0,
    require_location: bool = False,
    require_salary: bool = False,
    min_word_count: int = 20
) -> pd.DataFrame:
    """
    Custom filtering with specific criteria.
    
    Args:
        input_csv: Input CSV file
        output_csv: Output CSV file
        min_companies: Minimum number of companies required
        min_skills: Minimum number of skills required
        min_positions: Minimum number of positions required
        require_location: Must have location
        require_salary: Must have salary info
        min_word_count: Minimum word count in cleaned text
    
    Returns:
        Filtered DataFrame
    """
    
    logger.info("="*70)
    logger.info("CUSTOM FILTERING PIPELINE")
    logger.info("="*70)
    logger.info(f"Criteria:")
    logger.info(f"  - Minimum companies: {min_companies}")
    logger.info(f"  - Minimum skills: {min_skills}")
    logger.info(f"  - Minimum positions: {min_positions}")
    logger.info(f"  - Require location: {require_location}")
    logger.info(f"  - Require salary: {require_salary}")
    logger.info(f"  - Minimum words: {min_word_count}")
    logger.info("="*70 + "\n")
    
    df = pd.read_csv(input_csv)
    original_count = len(df)
    
    # Count features
    df['company_count'] = df['companies_extracted'].fillna('').apply(lambda x: len([c for c in str(x).split(',') if c.strip()]))
    df['skill_count'] = df['skills_extracted'].fillna('').apply(lambda x: len([s for s in str(x).split(',') if s.strip()]))
    df['position_count'] = df['positions_extracted'].fillna('').apply(lambda x: len([p for p in str(x).split(',') if p.strip()]))
    df['has_location'] = df['locations_extracted'].fillna('').apply(lambda x: len(str(x).strip()) > 0)
    df['has_salary'] = df['salary_info'].fillna('').apply(lambda x: len(str(x).strip()) > 0)
    df['word_count'] = df['cleaned_text'].fillna('').apply(lambda x: len(str(x).split()))
    
    # Apply filters
    mask = (
        (df['company_count'] >= min_companies) &
        (df['skill_count'] >= min_skills) &
        (df['position_count'] >= min_positions) &
        (df['word_count'] >= min_word_count)
    )
    
    if require_location:
        mask = mask & df['has_location']
    
    if require_salary:
        mask = mask & df['has_salary']
    
    filtered_df = df[mask].copy()
    
    # Select columns
    existing_columns = [col for col in FINAL_COLUMNS if col in df.columns]
    filtered_df = filtered_df[existing_columns]
    
    # Save
    filtered_df.to_csv(output_csv, index=False)
    
    logger.info(f"Filtered {len(filtered_df)}/{original_count} emails")
    logger.info(f"Saved to: {output_csv}\n")
    
    return filtered_df

# ===================================================================
# QUALITY ANALYSIS
# ===================================================================

def analyze_data_quality(csv_path: str):
    """Analyze quality of extracted data."""
    
    logger.info("="*70)
    logger.info("DATA QUALITY ANALYSIS")
    logger.info("="*70 + "\n")
    
    df = pd.read_csv(csv_path)
    
    logger.info(f"Total Emails: {len(df)}")
    logger.info(f"\nColumn Completeness:")
    
    for col in ['companies_extracted', 'skills_extracted', 'positions_extracted', 
                'locations_extracted', 'salary_info', 'experience_required', 'degrees_required']:
        if col in df.columns:
            non_empty = df[col].notna() & (df[col] != '')
            count = non_empty.sum()
            percentage = (count / len(df)) * 100
            logger.info(f"  {col}: {count} ({percentage:.1f}%)")
    
    logger.info("\n" + "="*70 + "\n")

# ===================================================================
# MAIN EXECUTION
# ===================================================================

if __name__ == "__main__":
    
    # Option 1: Standard filtering (recommended)
    logger.info("OPTION 1: STANDARD FILTERING")
    logger.info("Keep emails with at least 2 features extracted\n")
    
    filtered_df = filter_relevant_emails(
        input_csv="ai_cleaned_emails.csv",
        output_csv="relevant_placement_emails.csv"
    )
    
    # Option 2: Custom filtering (if you need specific criteria)
    # Uncomment and modify as needed:
    
    # logger.info("\n🔵 OPTION 2: CUSTOM FILTERING")
    # logger.info("Keep emails with specific requirements\n")
    # 
    # custom_df = filter_by_custom_criteria(
    #     input_csv="ai_cleaned_emails.csv",
    #     output_csv="custom_placement_emails.csv",
    #     min_companies=1,      # Must have at least 1 company
    #     min_skills=2,         # Must have at least 2 skills
    #     min_positions=1,      # Must have at least 1 position
    #     require_location=True, # Must have location
    #     require_salary=False,  # Salary optional
    #     min_word_count=30     # At least 30 words
    # )
    
    # Option 3: Quality analysis
    logger.info("\nANALYZING DATA QUALITY")
    analyze_data_quality("relevant_placement_emails.csv")
    
    logger.info("All filtering operations complete!")